## January 17, 2019 ##

According to Beck, traditional game theory says little about games of complete information like Hex. Trying to find an optimal strategy using backtracking is impractical due to "computational chaos" i.e. the exponential growth of the game tree. An alternative approach is the fake probabilistic method, which involves using randomization and then applying potential functions (derandomizing.) This approach is global and therefore useful for games that cannot be decomposed into smaller sub-games. Hex, chess, and tic-tac-toe fall into this category.

A "Weak Win" happens when a player achieves a winning configuration, but not necessarily first. A "Strong Draw" (and possibly an outright loss) happens when a player fails to achieve a Weak Win. Using probabilistic analysis on a randomized game (a game in which both players play randomly) can lead to an optimal Weak Win/Strong Draw strategy, hence the utility of the fake probabilistic method. This method works best on large inputs.

A hypergraph is a graph in which each edge can connect any number of vertices. Hex is played on an arbitrarily large hypergraph in which the vertices represent cells on the board and edges represent winning game states. I'm going to need some clarification on this point. Does an edge represent all game states that include a particular winning path?

Achievement and Avoidance Numbers describe the threshold between a Weak Win and a Strong Draw. This is also called the game theoretic breaking point. For Clique, the Achievement Number is the largest clique size a maker can build and the Avoidance Number is the smallest clique size a forcer can force a maker to build. How does this translate to Hex? 

The central idea of this section seems to be that players can force games that require global analysis to decompose into smaller, composite games. Understanding this process is one strategy for dealing with computational chaos.

Chapter 10 presents Conway's Solitaire Army proof and then introduces and proves the Erdos-Selfridge Theorem. The strategy used in the proof looks similar to the strategy we applied in the Hex program. What Erdos-Selfridge says is that the Achievement Number is always less than or equal to the Majority Play Number (the number obtained in the first part of the fake probabilistic method.) 

Miscellaneous questions:
What are "Ramseyish" games?
How do you visualize n^d torus tic-tac-toe?

References: 
J. Beck, *Combinatorial Games: Tic-Tac-Toe Theory,* New York: Cambridge University Press. 2008.

## January 22, 2019 ##

This week I worked through Beck's proof of the Erd&#337;s-Selfridge Theorem. 

First, some definitions. An $n$-uniform hypergraph is a hypergraph in which each edge connects $n$ nodes. In other words, it's a collection of sets of size $n$. The magnitude of $\mathcal{F}$ is the number of nodes in $\mathcal{F}$ and the maximum degree of $\mathcal{F}$ is the greatest number of sets (edges) that any one node belongs to. 

We require that $|\mathcal{F}| + MaxDeg(\mathcal{F}) < 2^n$. So it looks like we need the nodes to be fairly evenly distributed among the edges, with no one node belonging to too many edges. This makes sense; the more edges a node belongs to, the more strategically valuable it is. If the first player chooses a valuable-enough node in an unbalanced graph they could force a win.

With this in mind, we want to show that the second player can force a Strong Draw. The first part of the proof describes the potential function we implemented in the Hex program, only this time we're looking at the game from the second player's perspective. In the context of Hex, a "winning set" is a winning path for the first player. "Dead sets" are winning paths containing at least one cell that is already occupied by the second player. The sets we need to worry about are the "survivors." These are the remaining potential win states for the first player.

We define a function that describes the sum total "danger" of the current game state. This is exactly the function we used in our implementation. The second player chooses their next move to minimize the value of this function, meaning they choose the move that "kills" the most winning sets. Then, because of the way we defined the potential function, any move the first player makes doubles the "danger" of the remaining sets it belongs to. The key idea is that because of the strategy used by the second player, the first player cannot add back more "danger" than what was removed in the previous step. So the function is monotonically non-increasing.

For the first player to win, they must fully occupy a winning set. This would cause the total "danger" to be greater than or equal to 1. But we have that $|\mathcal{F}| + MaxDeg(\mathcal{F}) < 2^n$ and therefore $(|\mathcal{F}| + MaxDeg(\mathcal{F}))2^{-n} < 1$. But the term on the left side of this inequality must be greater than or equal to the "danger" of the starting state. So the danger of the starting state is less than 1. Combined with the fact that the function is monotonically non-increasing, this means that the first player cannot achieve such a state. Hence, the second player can force a Strong Draw.

References: 
J. Beck, *Combinatorial Games: Tic-Tac-Toe Theory,* New York: Cambridge University Press. 2008.

## January 29, 2019 ##

Apologies for posting this late. I was trying to understand page 300 of the Erd&#337;s-Selfridge paper in preparation for a presentation, but I couldn't figure out how they derived the $k > n\ log\ n$ bound for guaranteeing a second-player draw. I was trawling the internet for clues and found an amazing paper by a guy named Robert Gammill from the Rand Corporation. It was published in 1974, around the time the Erd&#337;s-Selfridge paper was published. In it, he examines how the Erd&#337;s-Selfridge theorem applies to tic-tac-toe. He uses the theorem to compute more precise bounds for forced-draw grames. The nice part is that it's quite straightforward to see how he derived them. Unfortunately, this hasn't brought me any closer to understanding the bound from the Erd&#337;s-Selfridge paper. 

The rest of the paper is devoted to implementing Qubic, the $4^3$ version of tic-tac-toe. It's a fun read. It sounds like they had a great time testing it.

References: 
J. Gammill. "An Examination of TIC-TAC-TOE like games," National Computer Conference, 1974. https://www.computer.org/csdl/proceedings/afips/1974/5082/00/50820349.pdf

## February 5, 2019 ##

This week I implemented a simple $4^2$ tic-tac-toe game that provides the second player with the information they need to play the potential strategy. I used an object-oriented design for readability and flexibility. If I were to keep developing it, there are a few things I would change. I would write a function to compute the winning paths instead of hardcoding them, I would compute Erdős-Selfridge potentials for both players, and I would make the input validation more robust. Also, since time was limited I didn't prioritize the the user interface. It could definitely be improved.

In [ ]:
import random


class Board:
    def __init__(self, n):
        self.cells = [[' ' for _ in range(n)] for _ in range(n)]
        
        self.winning_lines = []
        # add winning rows
        for i in range(n):
            self.winning_lines.append([])
            for j in range(n):
                self.winning_lines[-1].append((i, j))
                
        # add winning columns
        for i in range(n):
            self.winning_lines.append([])
            for j in range(n):
                self.winning_lines[-1].append((j, i))
                
        # add winning diagonal lines
        if n % 2 == 1:
            self.winning_lines.append([])
            for i in range(n):
                self.winning_lines[-1].append((i, i))
            self.winning_lines.append([])
            for i in range(n):
                self.winning_lines[-1].append(((n-1)-i, i))
                
        print(self.winning_lines)
        
    def __str__(self):
        result = '\n'
        for row in self.cells:
            result += str(row) + '\n'
        return result

    def get_open_cells(self):
        open_cells = []
        for i in range(len(self.cells)):
            for j in range(len(self.cells[i])):
                if self.cells[i][j] == ' ':
                    open_cells.append((i, j))
        return open_cells

    def compute_danger(self):
        # for simplicity, we only compute player x danger
        # = sum of (1/2)^(# of open cells remaining
        # in line that contains no o's)
        danger = 0
        for line in self.winning_lines:
            if 'o' not in line:
                danger += (1/2)**(len(line) - line.count('x'))
        return danger

    def get_optimal_move(self):
        # maximize the value we subtract from the "danger" function
        open_cells = self.get_open_cells()
        best_move = open_cells[0]
        best_move_val = 0

        for pos in open_cells:
            val = 0
            for line in self.winning_lines:
                if pos in line and 'o' not in line:
                    val += (1/2)**(len(line) - line.count('x'))
            if val > best_move_val:
                best_move = pos
                best_move_val = val

        return best_move

    def add_move(self, position, symbol):
        # in all lines, replace position with symbol
        for i in range(len(self.winning_lines)):
            for j in range(len(self.winning_lines[i])):
                if self.winning_lines[i][j] == position:
                    self.winning_lines[i][j] = symbol

        # remove winning lines that contain both symbols
        temp = list(filter(lambda lst: not('x' in lst and 'o' in lst), self.winning_lines))
        self.winning_lines = temp

        # add move to board
        self.cells[position[0]][position[1]] = symbol

    def check_for_win(self):
        if len(self.winning_lines) == 0:
            return True, 'No one'

        for line in self.winning_lines:
            if line.count('x') == len(line):
                return True, 'Player X'
            elif line.count('o') == len(line):
                return True, 'Player O'
        return False, None


class Player:
    def __init__(self, symbol, board):
        self.symbol = symbol
        self.board = board

    def move_randomly(self):
        open_cells = self.board.get_open_cells()
        random_index = random.randint(0, len(open_cells)-1)
        position = open_cells[random_index]
        self.board.add_move(position, self.symbol)

    def choose_move(self):
        while True:
            try:
                pos = input('Please enter x,y coordinates: ').split(',')
                pos = tuple(int(c.strip("()[] ")) for c in pos)
                if pos in self.board.get_open_cells():
                    self.board.add_move(pos, self.symbol)
                    break
                else:
                    print('Cell not available.')
            except:
                print('Cannot read input. Please try again.')


# main game
board = Board(3)
player_x = Player('x', board)
player_o = Player('o', board)
game_over = False
winner = None

while True:
    print(board)
    print('Player X, choose an option:')
    opt = int(input('(1) Enter a move, (2) Random move\n').strip())
    if opt == 1:
        player_x.choose_move()
    elif opt == 2:
        player_x.move_randomly()

    game_over, winner = board.check_for_win()
    if game_over:
        break

    potential = board.compute_danger()
    optimal_move = board.get_optimal_move()
    print(board)
    print('Erdos-Selfridge potential: ' + str(potential))
    print('Optimal move: ' + str(optimal_move) + '\n')

    print('Player O, choose an option:')
    opt = int(input('(1) Enter a move, (2) Random move\n').strip())
    if opt == 1:
        player_o.choose_move()
    elif opt == 2:
        player_o.move_randomly()

    game_over, winner = board.check_for_win()
    if game_over:
        break

for row in board.cells:
    print(row)
print('\nGame over. ' + str(winner) + ' wins!')


[[(0, 0), (0, 1), (0, 2)], [(1, 0), (1, 1), (1, 2)], [(2, 0), (2, 1), (2, 2)], [(0, 0), (1, 0), (2, 0)], [(0, 1), (1, 1), (2, 1)], [(0, 2), (1, 2), (2, 2)], [(0, 0), (1, 1), (2, 2)], [(2, 0), (1, 1), (0, 2)]]

[' ', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Player X, choose an option:
